In [134]:
from utils import *
import random
import csv
import time
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [130]:
def compute_query_score(query,utility_matrix,queries,usersID,k):
    queriesIDs=queries['queryID'].values.tolist()
    queries=queries['query'].values.tolist()
    #get a set of queries
    query_list=[]
    for i in queries:
        query={}
        for x in i.split(','):
            aux=x.split('=')
            query.update({aux[0]:aux[1]})
        query_list.append(query)
    #compute query similarity 
    values_similarity=[]
    for i in query_list:
        values_similarity.append(jaccard_similarity(query.values(),i.values()))
    values_similarity=np.array(values_similarity)
    queriesIDs=np.array(queriesIDs)
    #test if the query is already scored
    if values_similarity.max()==1:
        return utility_matrix[queriesIDs[values_similarity.argmax()]]
    else:
        #get the k nearest neighbor
        values_similarity,queriesIDs=KNN(k,values_similarity,queriesIDs)
        #compute scores
        scores=(np.array(utility_matrix[queriesIDs])@values_similarity)/values_similarity.sum()
    return pd.DataFrame(scores, index=usersID,columns=['Q2001'])

In [107]:
um= pd.read_csv('csv\\predicted_utility_matrix.csv',index_col=0)
queries=pd.read_csv("./csv/queries.csv",index_col=False)
queriesIDs=queries['queryID'].values.tolist()
queries=queries['query'].values.tolist()
users = pd.read_csv("./csv/users.csv",index_col=False)
usersID=users['Used_Id'].values.tolist()
dataset=pd.read_csv("./csv/dataset.csv",index_col=False)


In [108]:
def transfrom_query(query):
    output={}
    for x in query.split(','):
            aux=x.split('=')
            output.update({aux[0]:aux[1]})
    return output

In [114]:
columns=um.columns
queries_IDs_to_test = columns[np.random.choice(range(2000), size=(50), replace=False)]
queries_to_test=um[queries_IDs_to_test]
data_to_test_with=um.loc[:, ~um.columns.isin(queries_IDs_to_test)]
allqueries=pd.read_csv("./csv/queries.csv",index_col=False)
queriesID=data_to_test_with.columns
queries=allqueries[allqueries['queryID'].isin(queriesID)]
allaqueries=allqueries['query']

In [115]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=0
for Id in queries_IDs_to_test:
    query=transfrom_query(allaqueries[queriesIDs.index(Id)])
    df=compute_query_score(query,data_to_test_with,queries,usersID)
    print(Id,round(sqrt(mean_squared_error(np.array(df),np.array(queries_to_test[Id]))),2))
    rmse+=sqrt(mean_squared_error(np.array(df),np.array(queries_to_test[Id])))
rmse=rmse/2000

Q1569 45.66
Q24 24.18
Q1175 24.0
Q1817 24.2
Q191 30.9
Q1885 38.58
Q146 45.65
Q710 38.38
Q412 10.46
Q1474 38.81
Q299 25.26
Q1728 38.4
Q1843 23.9
Q842 51.46
Q705 51.42
Q760 47.98
Q1308 41.14
Q1865 48.15
Q153 23.79
Q1571 25.17
Q1355 24.13
Q1506 45.62
Q568 40.57
Q1087 23.74
Q518 45.94
Q253 41.31
Q919 51.29
Q1467 40.68
Q1113 30.71
Q719 10.46
Q1607 25.29
Q1585 25.27
Q1622 10.32
Q800 25.06
Q62 38.75
Q1301 51.13
Q87 40.94
Q1683 25.1
Q1243 30.46
Q1891 23.89
Q348 48.16
Q932 24.35
Q1495 24.14
Q1142 30.72
Q1544 38.68
Q754 45.61
Q1182 51.78
Q1729 10.17
Q1963 41.16
Q782 51.48
